In [6]:
import pandas as pd
import requests
import json

from datetime import datetime
import glob
from enum import Enum

from aind_data_schema.core.procedures import SpecimenProcedure, SpecimenProcedureType, Immunolabeling, HCRSeries


base = pd.read_excel("./ExM Sample Tracking smartsheet.xlsx", sheet_name="ExM Sample Tracking", header=[0], converters={})


c:\Users\mae.moninghoff\AppData\Local\miniconda3\envs\aind_data_migration_scripts\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
with pd.option_context("display.multi_sparse", False):
    print(base.columns)
    

Index(['Auto', 'Submitted Priority', 'TRUE Priority', '#', 'External Sample',
       'Status', 'Processing Lead', 'Sample', 'Genotype', 'Label',
       'Age of Injection', 'tTA Virus Dose', 'DCM Delipidation Start',
       'DCM Delipidation End', 'SBiP Delipidation Start',
       'SBiP Delipidation End', 'Immuno: Primary Antibody',
       'Mass of Primary Antibody used per Brain (ug)',
       'Primary Antibody Catalog #', 'Primary Antibody Lot #',
       'Primary Antibody RRID', 'Immuno: Primary Ab Start Date',
       'Immuno Secondary', 'Mass of Secondary Antibody used per Brain (ug)',
       'Secondary Antibody Catalog #', 'Secondary Antibody Lot #',
       'Secondary Antibody RRID', 'Fluorophore',
       'Immuno: Secondary Ab Start Date', 'Gelation: MBS Start',
       'Gelation: MBS End', 'Gelation: AcX Start', 'Gelation: AcX End',
       'Gelation: PBS Wash Start', 'Gelation: PBS Wash End',
       'Gelation: Stock X + VA-044 Equilibration  Start',
       'Gelation: Stock X + VA-044

In [17]:
subj_procedures = {}

download_files = True


if download_files:
    for row_idx, row in base.iterrows():    

        sample_id = str(row['Sample'])
        print(sample_id)
        
        if len(sample_id) != 6:
            continue

        # INFO: This originally loaded in the procedures files, but is not necessary at this moment since i have them saved. Prior block now loads these files into same structure.
        request = requests.get(f"http://aind-metadata-service/procedures/{sample_id}")
        #TODO: check for invalid/missing models

        if request.status_code == 404:
            print(f"{sample_id} model not found")

        item = request.json()
        print(item['message'])

        if item['message'] == 'Valid Model.':
            with open(f'./original_spec_files/{sample_id}.json', 'w') as outfile:
                json.dump(item['data'], outfile)
        else:
            with open(f'./original_spec_files/{sample_id}_invalid.json', 'w') as outfile:
                json.dump(item['data'], outfile)

576404
Validation Errors: 1 validation error for Procedures
subject_procedures.0.Surgery.procedures.0.Perfusion.protocol_id
  Field required [type=missing, input_value={'procedure_type': 'Perfu...ecimen_ids': {'576404'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
609105
Validation Errors: 5 validation errors for Procedures
subject_procedures.0.Surgery.procedures.0.Retro-orbital injection.injection_materials
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.6/v/too_short
subject_procedures.0.Surgery.procedures.0.Retro-orbital injection.protocol_id
  Field required [type=missing, input_value={'injection_materials': [..., 'injection_eye': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
subject_procedures.0.Surgery.procedures.0.Retro-orbital injection.injection_volume

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
class SpecMaker():
    def __init__(self, experimenter, sample_id, existing_procedure:dict=None) -> None:
        self.experimenter = experimenter
        self.sample_id = sample_id
        self.existing_procedure = existing_procedure
        
    def make_spec_procedure(self, start, end, procedure_type, procedure_name, notes=None, protocol_id="unknown", reagent=[], immunolabeling:Immunolabeling=None, hcr_series:HCRSeries=None):
        return SpecimenProcedure(
            procedure_type=procedure_type,
            procedure_name=procedure_name,
            specimen_id=self.sample_id,
            start_date=start,
            end_date=end,
            experimenter_full_name=self.experimenter,
            protocol_id=protocol_id,
            reagents=reagent,
            hcr_series=hcr_series,
            immunolabeling=immunolabeling,
            notes=notes,
        )
        

In [5]:


def basic_dict(name, start, end, type) -> dict:
        return {
            "procedure_name": name,
            "start_date": start,
            "end_date": end,
            "procedure_type": type,
        }

for row_idx, row in base.iterrows():

    items = []

    
    # general info
    processor = row['Processing Lead']
    sample_id = str(row['Sample'])
    genotype = row['Genotype']
    label = row['Label']
    inj_age = row['Age of Injection']
    tta_virus_dose = row['tTA Virus Dose']

    spec_maker = SpecMaker(processor, sample_id)

      
    
    
    mbs_start = row['Gelation: MBS Start']
    mbs_end = row['Gelation: MBS End']
    acx_start = row['Gelation: AcX Start']
    acx_end = row['Gelation: AcX End']
    pbs_start = row['Gelation: PBS Wash Start']
    pbs_end = row['Gelation: PBS Wash End']
    stock_start = row['Gelation: Stock X + VA-044 Equilibration  Start']
    stock_end = row['Gelation: Stock X + VA-044 Equilibration End']
    prok_rt_start = row['Gelation +  ProK RT Start']
    prok_rt_end = row['Gelation +  ProK RT End']
    prok_rt_time = row['Duration (days)']
    prok_37c_start = row['Gelation + Add\'l ProK 37C Start']
    prok_37c_end = row['Gelation + Add\'l ProK 37C End']
    prok_37c_time = row['Duration (days) Gelation + Add\'l ProK 37C 37C']
    pbs2_start = row['PBS Wash Start']
    pbs2_end = row['PBS Wash End']
    pbs_stored = row['Date of Storage in PBS Az 0.05% @4C']
    expansion_start = row['Expansion Start Date']
    expansion_end = row['Expansion End Date']
    notes1 = row['Notes']
    notes2 = row['Notes II']
    notes3 = row['Notes III']

    # delipidation
    dcm_delip_name = 'DCM Delipidation'
    dcm_delip_start = row['DCM Delipidation Start']
    dcm_delip_end = row['DCM Delipidation End']
    items.append(basic_dict(dcm_delip_name, dcm_delip_start, dcm_delip_end, SpecimenProcedureType.DELIPIDATION))


    sbip_delip_name = 'SBiP Delipidation'
    sbip_delip_start = row['SBiP Delipidation Start']
    sbip_delip_end = row['SBiP Delipidation End']
    items.append(basic_dict(sbip_delip_name, sbip_delip_start, sbip_delip_end, SpecimenProcedureType.DELIPIDATION))

    # immunolabeling
    immunolabeling_name = 'Immunolabeling'
    immuno_primary_antibody = row['Immuno: Primary Antibody']
    immuno_secondary_antibody = row['Immunolabeling (31-33d)']['Unnamed: 14_level_1']['Secondary (20ug)']
    # items.append(basic_dict(immunolabeling_name, immunolabeling_primary, immunolabeling_secondary, SpecimenProcedureType.IMMUNOLABELING))
    immuno_primary_mass = row['Mass of Primary Antibody used per Brain (ug)']
    immuno_primary_catalogue = row['Primary Antibody Catalog #']
    immuno_primary_lot = row['Primary Antibody Lot #']
    immuno_primary_rrid = row['Primary Antibody RRID']
    immuno_primary_start = row['Immuno: Primary Ab Start Date']
    immuno_secondary_antibody = row['Immuno Secondary']
    immuno_secondary_mass = row['Mass of Secondary Antibody used per Brain (ug)']
    immuno_secondary_catalogue = row['Secondary Antibody Catalog #']
    immuno_secondary_lot = row['Secondary Antibody Lot #']
    immuno_secondary_rrid = row['Secondary Antibody RRID']
    fluorophore = row['Fluorophore']
    immuno_secondary_start = row['Immuno: Secondary Ab Start Date']


    specimen_procedures = []
    for item in items:
        print(item)
        if pd.isna(item['start_date']) or pd.isna(item['end_date']):
            continue
        specimen_procedures.append(spec_maker.make_spec_procedure(item['start_date'], item['end_date'], item['procedure_type'], item['procedure_name']))
    
    print(specimen_procedures)



KeyError: 'Processed By:'